In [ ]:
!pip install einops
!pip install kornia
!pip install git+https://github.com/cvg/LightGlue.git

In [ ]:
import os
import glob

from lightglue import LightGlue, SuperPoint, DISK
from lightglue.utils import load_image, match_pair
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
sp = SuperPoint(max_num_keypoints=4096).eval().cuda()
matcher_sp = LightGlue(pretested='superpoint').eval().cuda()

size = 640

In [ ]:
train_df = pd.read_csv('/kaggle/input/csc-hackathon-2023-lunua-task/train.csv')
train_df['image_name1'] = train_df['image_url1'].apply(lambda x: os.path.basename(x))
train_df['image_name2'] = train_df['image_url2'].apply(lambda x: os.path.basename(x))

train_df['sp_min'] = -1
train_df['sp_sqrt'] = -1

for ind in tqdm(train_df.index):
    try:
        image_1, scales_1 = load_image(f"/kaggle/input/csc-hackathon-2023-lun/train/{train_df.loc[ind, 'image_name1']}", resize=size)
        image_2, scales_2 = load_image(f"/kaggle/input/csc-hackathon-2023-lun/train/{train_df.loc[ind, 'image_name2']}", resize=size)

        image_1, scales_1 = image_1.cuda(), scales_1.cuda()
        image_2, scales_2 = image_2.cuda(), scales_2.cuda()

        pred_sp = match_pair(sp, matcher_sp, image_1, image_2, scales0=scales_1, scales1=scales_2)
        train_df.loc[ind, 'sp_min'] = pred_sp['matching_scores'].shape[0] / min(pred_sp['keypoints0'].shape[0], pred_sp['keypoints1'].shape[0])
        train_df.loc[ind, 'sp_sqrt'] = pred_sp['matching_scores'].shape[0] / np.sqrt(pred_sp['keypoints0'].shape[0] * pred_sp['keypoints1'].shape[0])

    except:
        continue

train_df.to_csv('train_local_features.csv')

In [ ]:
test_df = pd.read_csv('/kaggle/input/csc-hackathon-2023-lunua-task/test-data.csv')
test_df['image_name1'] = test_df['image_url1'].apply(lambda x: os.path.basename(x))
test_df['image_name2'] = test_df['image_url2'].apply(lambda x: os.path.basename(x))

test_df['sp_min'] = -1
test_df['sp_sqrt'] = -1

for ind in tqdm(test_df.index):
    try:
        image_1, scales_1 = load_image(f"/kaggle/input/csc-hackathon-2023-lun/test/{test_df.loc[ind, 'image_name1']}", resize=size)
        image_2, scales_2 = load_image(f"/kaggle/input/csc-hackathon-2023-lun/test/{test_df.loc[ind, 'image_name2']}", resize=size)

        image_1, scales_1 = image_1.cuda(), scales_1.cuda()
        image_2, scales_2 = image_2.cuda(), scales_2.cuda()

        pred_sp = match_pair(sp, matcher_sp, image_1, image_2, scales0=scales_1, scales1=scales_2)
        test_df.loc[ind, 'sp_min'] = pred_sp['matching_scores'].shape[0] / min(pred_sp['keypoints0'].shape[0], pred_sp['keypoints1'].shape[0])
        test_df.loc[ind, 'sp_sqrt'] = pred_sp['matching_scores'].shape[0] / np.sqrt(pred_sp['keypoints0'].shape[0] * pred_sp['keypoints1'].shape[0])

    except:
        continue

test_df.to_csv('test_local_features.csv')